# Evaluating model and interpret the classification results

In [68]:
import torchray
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
from torchray.benchmark import get_example_data, plot_example
from torchvision import models
import torchvision

In [36]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.Resize(224),
        #         transforms.CenterCrop(800),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir)
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes


In [37]:
Path('../data/') / "test"

PosixPath('../data/test')

In [39]:
data_prep = DataPreparation('../data/figures/test')
data, size = data_prep.create_dataloaders(1, False, 4)

In [73]:
model_path = Path().cwd().parents[0] / "models/resnet50_d_26_t_10:07.pth"

In [76]:
model = torch.load(model_path)

In [96]:
infer = model.eval();

In [ ]:
# Grad-CAM backprop.
saliency_layer = get_module(model, 'features.29')

probe = Probe(saliency_layer, target='output')

y = model(x)
z = y[0, category_id]
z.backward()

saliency = gradient_to_grad_cam_saliency(probe.data[0])

# Plots.
plot_example(x, saliency, 'grad-cam backprop', category_id)
